In [ ]:
import torch # to create tensors to store the raw data, weights and biases 
import torch.nn as nn # to make the weights and biases learnable (part of the network)
import torch.nn.functional as F # to apply activation functions 
from torch.optim import Adam  
from torch.distributions.uniform import Uniform

import lightning as L # to train the model 
from torch.utils.data import DataLoader, TensorDataset # to load the data 
from lightning.pytorch.tuner.tuning import Tuner      
from lightning.pytorch.callbacks import ModelCheckpoint #Lightning is trying to delete the previous checkpoint (because the default ModelCheckpoint is set to save_top_k=1).
                                                        # On Windows a file gets locked as soon as any program (Explorer preview, antivirus, TensorBoard, VS Code, …) opens it, and Windows then blocks the delete call ⇒ PermissionError WinError 32

import pandas as pd # to load the data 

import matplotlib.pyplot as plt # graphs 
import seaborn as sns # graphs


In [ ]:
inputs= torch.tensor([1,0,0,0],
                    [0,1,0,0],
                    [0,0,1,0],
                    [0,0,0,1])
labels= torch.tensor([0,1,0,0],
                    [0,0,1,0],
                    [0,0,0,1],
                    [0,1,0,0])

dataset= TensorDataset(inputs,labels)
dataloader= DataLoader(dataset)

In [ ]:
class WordEmbeddingFromScratch(LightningModule):
    def __init__(self):
        super().__init__()
        
    def forward(self, input):
        return input
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)
    def training_step(self, batch, batch_idx):
        return torch.tensor(0.0)
